# Evolver Loop 1 Analysis

Analyzing the current state and planning next experiments.

In [1]:
import pandas as pd
import numpy as np
import os
import json

# Load session state
with open('/home/code/session_state.json', 'r') as f:
    state = json.load(f)

print("Current Experiments:")
for exp in state['experiments']:
    print(f"  {exp['name']}: CV={exp['cv_score']:.6f}, LB={exp.get('lb_score', 'N/A')}")

print(f"\nTarget: 68.919154")
print(f"Best CV: {min(exp['cv_score'] for exp in state['experiments']):.6f}")
print(f"Gap: {min(exp['cv_score'] for exp in state['experiments']) - 68.919154:.6f}")

Current Experiments:
  001_baseline: CV=70.676102, LB=None

Target: 68.919154
Best CV: 70.676102
Gap: 1.756948


In [2]:
# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

# Calculate bounding box for a single tree at different angles
def get_bbox_side(angle_deg):
    angle_rad = np.radians(angle_deg)
    cos_a = np.cos(angle_rad)
    sin_a = np.sin(angle_rad)
    
    # Rotate vertices
    x_rot = [cos_a * x - sin_a * y for x, y in zip(TX, TY)]
    y_rot = [sin_a * x + cos_a * y for x, y in zip(TX, TY)]
    
    # Bounding box
    side = max(max(x_rot) - min(x_rot), max(y_rot) - min(y_rot))
    return side

# Find optimal angle for N=1
angles = np.arange(0, 360, 0.001)
sides = [get_bbox_side(a) for a in angles]
best_idx = np.argmin(sides)
print(f"Optimal angle for N=1: {angles[best_idx]:.3f} degrees")
print(f"Optimal side length: {sides[best_idx]:.6f}")
print(f"Score contribution: {sides[best_idx]**2:.6f}")

Optimal angle for N=1: 45.000 degrees
Optimal side length: 0.813173
Score contribution: 0.661250


In [3]:
# Check current N=1 configuration
preopt_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025-csv/santa-2025.csv'
df = pd.read_csv(preopt_path)

# Parse N=1
n1_row = df[df['id'] == '001_0'].iloc[0]
x = float(str(n1_row['x']).replace('s', ''))
y = float(str(n1_row['y']).replace('s', ''))
deg = float(str(n1_row['deg']).replace('s', ''))

print(f"Current N=1 configuration:")
print(f"  x={x}, y={y}, deg={deg}")
print(f"  Side length: {get_bbox_side(deg):.6f}")
print(f"  Score contribution: {get_bbox_side(deg)**2:.6f}")

Current N=1 configuration:
  x=-48.196086194214246, y=58.770984615214225, deg=45.0
  Side length: 0.813173
  Score contribution: 0.661250


In [4]:
# Check if 45 degrees is optimal
print("\nComparing angles around 45 degrees:")
for angle in [44.5, 44.75, 45.0, 45.25, 45.5]:
    side = get_bbox_side(angle)
    print(f"  {angle:.2f}°: side={side:.6f}, score={side**2:.6f}")


Comparing angles around 45 degrees:
  44.50°: side=0.815919, score=0.665723
  44.75°: side=0.814553, score=0.663497
  45.00°: side=0.813173, score=0.661250
  45.25°: side=0.814553, score=0.663497
  45.50°: side=0.815919, score=0.665723


In [5]:
# Analyze per-N scores from the baseline
from shapely.geometry import Polygon
from shapely import affinity

TREE_VERTICES = list(zip(TX, TY))

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = float(center_x)
        self.center_y = float(center_y)
        self.angle = float(angle)
        
        initial_polygon = Polygon(TREE_VERTICES)
        rotated = affinity.rotate(initial_polygon, self.angle, origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=self.center_x, yoff=self.center_y)

def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return val[1:]
    return str(val)

def load_trees_for_n(df, n):
    prefix = f"{n:03d}_"
    rows = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in rows.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        trees.append(ChristmasTree(x, y, deg))
    return trees

def compute_bounding_side(trees):
    all_points = []
    for tree in trees:
        coords = np.array(tree.polygon.exterior.coords)
        all_points.append(coords)
    all_points = np.vstack(all_points)
    side = max(all_points.max(axis=0) - all_points.min(axis=0))
    return side

print("Per-N analysis (top 20 contributors):")
per_n_scores = []
for n in range(1, 201):
    trees = load_trees_for_n(df, n)
    side = compute_bounding_side(trees)
    contribution = side**2 / n
    per_n_scores.append({'n': n, 'side': side, 'contribution': contribution, 'efficiency': n / (side**2)})

per_n_df = pd.DataFrame(per_n_scores)
per_n_df = per_n_df.sort_values('contribution', ascending=False)
print(per_n_df.head(20).to_string())

Per-N analysis (top 20 contributors):


     n      side  contribution  efficiency
0    1  0.813173      0.661250    1.512287
1    2  0.949504      0.450779    2.218381
2    3  1.142031      0.434745    2.300198
4    5  1.443692      0.416850    2.398947
3    4  1.290806      0.416545    2.400702
6    7  1.673104      0.399897    2.500647
5    6  1.548438      0.399610    2.502438
8    9  1.867280      0.387415    2.581211
7    8  1.755921      0.385407    2.594658
14  15  2.384962      0.379203    2.637111
9   10  1.940696      0.376630    2.655125
20  21  2.811667      0.376451    2.656389
19  20  2.742469      0.376057    2.659172
10  11  2.033002      0.375736    2.661443
21  22  2.873270      0.375258    2.664832
15  16  2.446640      0.374128    2.672882
25  26  3.118320      0.373997    2.673819
11  12  2.114873      0.372724    2.682950
12  13  2.200046      0.372323    2.685839
24  25  3.050182      0.372144    2.687129


In [6]:
# Summary
print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"Target score: 68.919154")
print(f"Current best: 70.676102")
print(f"Gap: 1.756948 (2.55%)")
print("\nKey insights from previous experiments:")
print("1. Pre-optimized santa-2025.csv is at a VERY TIGHT local optimum")
print("2. Standard optimizers (SA, backward propagation) found NO improvements")
print("3. Small N values (1-10) have highest score contribution")
print("4. N=1 is already at optimal 45-degree angle")
print("\nNext steps to try:")
print("1. Fractional translation (micro-adjustments at 0.001-0.00001 scale)")
print("2. Lattice-based packing for large N (72, 100, 144, 196, 200)")
print("3. Much longer optimization runs (hours, not minutes)")
print("4. Population-based optimization with perturbation")


SUMMARY
Target score: 68.919154
Current best: 70.676102
Gap: 1.756948 (2.55%)

Key insights from previous experiments:
1. Pre-optimized santa-2025.csv is at a VERY TIGHT local optimum
2. Standard optimizers (SA, backward propagation) found NO improvements
3. Small N values (1-10) have highest score contribution
4. N=1 is already at optimal 45-degree angle

Next steps to try:
1. Fractional translation (micro-adjustments at 0.001-0.00001 scale)
2. Lattice-based packing for large N (72, 100, 144, 196, 200)
3. Much longer optimization runs (hours, not minutes)
4. Population-based optimization with perturbation
